In [ ]:
!nvidia-smi

Thu Mar 30 03:09:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re
import copy

In [ ]:
!cp -r "/content/gdrive/My Drive/EVA8_S11_Course_Docs/GPT" "."

In [ ]:
%cd /content/GPT

/content/GPT


In [ ]:
import model_p
import utils_p
from model_p import Transformer
from utils_p import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    encode,
    decode,
    get_batch,
    save_model_to_chekpoint,
    estimate_loss,
)
print(f'    BATCH_SIZE : {BATCH_SIZE}, \
    BLOCK_SIZE : {BLOCK_SIZE}, \
    DEVICE : {DEVICE}, \
    DROPOUT : {DROPOUT}, \
    LEARNING_RATE : {LEARNING_RATE}')
print(f'NUM_EMBED : {NUM_EMBED}, \
    NUM_HEAD : {NUM_HEAD}, \
    NUM_LAYER : {NUM_LAYER}, \
    MAX_ITER : {MAX_ITER},\
    EVAL_INTER {EVAL_INTER}')

    BATCH_SIZE : 2,     BLOCK_SIZE : 20,     DEVICE : cuda,     DROPOUT : 0.2,     LEARNING_RATE : 0.0003
NUM_EMBED : 768,     NUM_HEAD : 6,     NUM_LAYER : 3,     MAX_ITER : 5,    EVAL_INTER 2


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

In [ ]:
# load model from checkpoint
# m = load_model_from_checkpoint(Transformer,vocab_size=vocab_size)

# example to decode sequence
# enc_sec = m.generate(idx=torch.zeros((1,1), dtype=torch.long),
# max_new_tokens=20)[0].tolist()
# print(decode(vocab=vocab, enc_sec=enc_sec))

In [ ]:
# raw data
path_do_data = "./data/english_small.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
print(f'len of data_raw : {len(data_raw)}')
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
print(vocab_size)
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
print(f'len(data) : {len(data)}')
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
print(f'len(train_data) : {len(train_data)}')
val_data = data[n:]
print(f'len(val_data) : {len(val_data)}')

# train a new model
model = Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
)

# load model to GPU if available

len of data_raw : 3269


Token indices sequence length is longer than the specified maximum sequence length for this model (722 > 512). Running this sequence through the model will result in indexing errors


30522
 encode -> len(text_seq) : 3269 len(tokens) : 722
 encode -> len(token_indices) : 722
 encode -> token_indices.shape torch.Size([722])
len(data) : 722
len(train_data) : 649
len(val_data) : 73
 TransformerBlock init : num_embed : 768, block_size : 20, num_heads : 6
 TransformerBlock init : num_embed : 768, block_size : 20, num_heads : 6
 TransformerBlock init : num_embed : 768, block_size : 20, num_heads : 6
 Transformer init self.vocab_size : 30522, self.num_embed : 768                  self.block_size : 20, self.num_heads : 6,                  self.num_layers : 3


In [ ]:
# load model to GPU if available
m = model.to('cuda')
# print the number of parameters in the model
print(
    "Model with {:.2f}M| parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)

Model with 68.19M| parameters


In [ ]:
print(m)

Transformer(
  (token_embedding_table): Embedding(30522, 768)
  (position_embedding_table): Embedding(20, 768)
  (blocks): Sequential(
    (0): TransformerBlock(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0): AttentionHead(
            (key): Linear(in_features=768, out_features=128, bias=False)
            (query): Linear(in_features=768, out_features=128, bias=False)
            (value): Linear(in_features=768, out_features=128, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (1): AttentionHead(
            (key): Linear(in_features=768, out_features=128, bias=False)
            (query): Linear(in_features=768, out_features=128, bias=False)
            (value): Linear(in_features=768, out_features=128, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (2): AttentionHead(
            (key): Linear(in_features=768, out_features=128, bias=False)
            (query): Linear(in_feature

In [ ]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
MAX_ITER = 3
for step in range(MAX_ITER):
    print(f'***********step : {step}**************')
    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        print(f' Estimating Train losses - EVAL_INTER : {EVAL_INTER }')
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print(f' EstimatingVal losses')
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    print(f' Resuming training with backprop')
    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()

***********step : 0**************
 Estimating Train losses - EVAL_INTER : 2
Estimate_loss model.eval() - len(data) : 649
Estimate_loss model.eval() - k : 0
get_batch ix.shape : torch.Size([2]), len(data) : 649
 Transformer fwd idx.shape : torch.Size([2, 20])
 Transformer fwd token_emb.shape : torch.Size([2, 20, 768])
 Transformer fwd posit_emb.shape : torch.Size([20, 768])
 Transformer fwd x.shape after token_emb + posit_emb : torch.Size([2, 20, 768])
    TransformerBlock fwd after MHA x.shape : torch.Size([2, 20, 768])
    TransformerBlock fwd after FFWD x.shape : torch.Size([2, 20, 768])
    TransformerBlock fwd after MHA x.shape : torch.Size([2, 20, 768])
    TransformerBlock fwd after FFWD x.shape : torch.Size([2, 20, 768])
    TransformerBlock fwd after MHA x.shape : torch.Size([2, 20, 768])
    TransformerBlock fwd after FFWD x.shape : torch.Size([2, 20, 768])
 Transformer fwd x.shape after TransformerBlock : torch.Size([2, 20, 768])
 Transformer fwd logits.shape after self.lm_he

In [ ]:
save_model_to_chekpoint(model=m, path_to_checkpoint="checkpoint", epoch=step)

In [ ]:
# generate some output based on the context
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(
    decode(
        enc_sec=m.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0],
        tokenizer=tokenizer,
    )
)